<a href="https://colab.research.google.com/github/winterForestStump/thesis/blob/main/Thesis_Llama_2_7B_and_LangChain_step_2_tables_ver3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Step 1: Install All the Required Packages and Libraries**

In [1]:
%%capture
!pip install huggingface_hub
!pip install langchain
!pip install sentence-transformers
!pip install chromadb
!pip install -q -U transformers peft accelerate optimum
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/
!pip install GPUtil

In [2]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
import torch
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from textwrap import fill
from huggingface_hub import hf_hub_download
import GPUtil
import requests

#**Step 1A: Recursively split by character**

https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter

In [3]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!nvidia-smi

Tue Jan 16 15:50:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
print('Initial GPU Usage:')
GPUtil.showUtilization()

Initial GPU Usage:
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |


In [5]:
text = requests.get('https://raw.githubusercontent.com/winterForestStump/thesis/main/data/txt_examp_ver2.txt').text

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

In [7]:
print('GPU Usage after text splitting:')
GPUtil.showUtilization()

GPU Usage after text splitting:
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |


In [8]:
texts = text_splitter.create_documents([text])

In [9]:
len(texts)

539

In [10]:
print(texts[0])
print(texts[1])

page_content='2969 Air Products & Chemicals, Inc. 10-K 2023-11-16 2023-09-30 2810 DE PA 0930 https://www.sec.gov/Archives/edgar/data/2969/0000002969-23-000047-index.html https://www.sec.gov/Archives/edgar/data/2969/000000296923000047/apd-20230930.htm https://www.sec.gov/Archives/edgar/data/2969/0000002969-23-000047.txt 2969_10K_2023_0000002969-23-000047.htm Item 1. Business\r\nAs used in this report, unless the context indicates otherwise, the terms �we,� �our,� �us,� the �Company,� "Air Products," or �registrant� include controlled subsidiaries and affiliates of Air Products.\r\nAbout Air Products'
page_content='About Air Products\r\nAir Products and Chemicals, Inc., a Delaware corporation originally founded in 1940, is a world-leading industrial gases company that has built a reputation for its innovative culture, operational excellence, and commitment to safety and the environment. Focused on serving energy, environmental, and emerging markets, we offer a portfolio of products and s

#**Step 1B: Embedding**

In [11]:
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

#query_result = embeddings.embed_query(texts[0].page_content)
#print(len(query_result))

In [12]:
print('GPU Usage after embedding:')
GPUtil.showUtilization()

GPU Usage after embedding:
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |


In [13]:
db = Chroma.from_documents(texts, embeddings, persist_directory="db")
results = db.similarity_search("What was the revenue of the company in 2023 and how it changed since 2021?", k=10)
print(results[0].page_content)

2023 IN SUMMARY
In fiscal year 2023, we achieved earnings growth through pricing discipline in our merchant business as well as improved on-site volumes, including higher demand for hydrogen, despite inflation, higher maintenance activities, and higher costs to support our long-term strategy. Due to the structure of our contracts, which generally contain fixed monthly charges and/or minimum purchase requirements, our on-site business generates stable cash flow and consistently contributes about half our total sales, regardless of the economic environment. We also recognized higher income from our equity affiliates due to the contribution of the second phase of the Jazan gasification and power project and positive results from other unconsolidated joint ventures across the regions.


In [14]:
print('GPU Usage after chroma DB:')
GPUtil.showUtilization()

GPU Usage after chroma DB:
| ID | GPU | MEM |
------------------
|  0 |  0% | 29% |


#**Step 2: Download the Model**

VER 1

In [15]:
MODEL_NAME = "TheBloke/Llama-2-7b-Chat-GPTQ"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto"
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [16]:
print('GPU Usage after LLM and tokenizer initialization:')
GPUtil.showUtilization()

GPU Usage after LLM and tokenizer initialization:
| ID | GPU | MEM |
------------------
|  0 |  3% | 36% |


Creating a plot template

In [17]:
template = """
<s>[INST] <<SYS>>
Act as a financial analitics expert. Use the following information to answer the question at the end. If you do not know the answer do not imagine it but just answer "I do not know".
<</SYS>>

{context}

{question} [/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])


qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 10}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

result = qa_chain(
    "What was the revenue (sales) of the company in 2023 and how it changed since 2021?"
)
print(fill(result["result"].strip(), width=90))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


According to the text, the company's sales in 2023 were $889 million, down 8% or $81.8
million from 2022. In comparison, the company's sales in 2021 were $3,895.8 million.
Therefore, the company's revenue decreased by $2,006.2 million between 2021 and 2023.


In [18]:
print('GPU Usage after the retrieval 1st prompt:')
GPUtil.showUtilization()

GPU Usage after the retrieval 1st prompt:
| ID | GPU | MEM |
------------------
|  0 | 98% | 53% |


other prompts:

In [19]:
result = qa_chain(
    "Summarize the financial results of the company in 2023 in 10-12 sentences."
)
print(fill(result["result"].strip(), width=90))

In 2023, the company achieved earnings growth through disciplined pricing in its merchant
business and improved on-site volumes, including higher demand for hydrogen despite
inflation, higher maintenance activities, and higher costs supporting its long-term
strategy. The company's on-site business generated stable cash flow and contributed
approximately half of its total sales, regardless of the economic environment.
Additionally, the company recognized higher income from its equity affiliates due to the
contribution of the second phase of the Jazan gasification and power project and positive
results from other unconsolidated joint ventures across regions. The company invested
$665.1 million in financing receivables, primarily for progress payments towards the
purchase of a natural gas-to-syngas processing facility in Uzbekistan. Overall, the
company's financial performance in 2023 was satisfactory, with earnings growth and
increased contributions from its on-site business and equity a

In [20]:
print('GPU Usage after the retrieval 2st prompt:')
GPUtil.showUtilization()

GPU Usage after the retrieval 2st prompt:
| ID | GPU | MEM |
------------------
|  0 | 91% | 53% |


In [21]:
result = qa_chain(
    "What were the net incomes from continuing operations of the company in the period from 2021 to 2023?"
)
print(fill(result["result"].strip(), width=90))

Based on the information provided in the expert's response, the net income from continuing
operations of the company in each of the periods from 2021 to 2023 are as follows: * For
the fiscal year ended 30 September 2021, the net income from continuing operations was
$2,028.8 million. * For the fiscal year ended 30 September 2022, the net income from
continuing operations was $2,243.5 million. * For the fiscal year ended 30 September 2023,
the net income from continuing operations was $2,266.5 million.


In [22]:
print('GPU Usage after the retrieval 3st prompt:')
GPUtil.showUtilization()

GPU Usage after the retrieval 3st prompt:
| ID | GPU | MEM |
------------------
|  0 | 98% | 73% |


In [23]:
torch.cuda.empty_cache()

In [24]:
print('GPU Usage after empting the cache:')
GPUtil.showUtilization()

GPU Usage after empting the cache:
| ID | GPU | MEM |
------------------
|  0 | 98% | 36% |


In [25]:
result = qa_chain(
    "What were earnings per share (EPS) in 2022-2023 period?"
)
print(fill(result["result"].strip(), width=90))

Based on the provided information, the earnings per share (EPS) in the 2022-2023 period
was $10.30.


In [26]:
print('GPU Usage after the retrieval 4th prompt:')
GPUtil.showUtilization()

GPU Usage after the retrieval 4th prompt:
| ID | GPU | MEM |
------------------
|  0 | 88% | 71% |


In [27]:
result = qa_chain(
    "What was the difference between total current assets and total current liabilities in 2023?"
)
print(fill(result["result"].strip(), width=90))

Based on the provided information, total current assets were $1,229.9 million in 2023,
while total current liabilities were $2,890.1 million. Therefore, the difference between
total current assets and total current liabilities in 2023 was $1,660.2 million ($2,890.1
- $1,229.9).
